In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups()

In [3]:
newsgroups.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [4]:
#print(newsgroups.DESCR)

In [5]:
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [6]:
print(newsgroups.data[0])

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [7]:
newsgroups.target_names[newsgroups.target[0]]

'rec.autos'

In [8]:
categories = ['talk.politics.guns', 'talk.religion.misc', 'comp.graphics', 'rec.autos']
newsgroups = fetch_20newsgroups(categories=categories)
target_names = newsgroups.target_names
#newsgroups_train = fetch_20newsgroups(categories=categories, subset='train')
#newsgroups_test = fetch_20newsgroups(categories=categories, subset='test')

In [9]:
#newsgroups_train.target.shape

In [10]:
#newsgroups_test.target.shape

In [11]:
print(newsgroups.data[0])

From: HADCRJAM@admin.uh.edu (MILLER, JIMMY A.)
Subject: Re: BATF/FBI revenge
Organization: University of Houston Administrative Computing
Lines: 38
Distribution: world
NNTP-Posting-Host: uhad2.admin.uh.edu
X-News-Reader: VMS NEWS 1.24
In-Reply-To: donb@netcom.com's message of Tue, 20 Apr 1993 17:10:52 GMT

In <donbC5sL24.Ewu@netcom.com> donb@netcom.com writes:

> Anyway, here's how I see the Waco affair; I'd be interested in other peoples'
> interpretations...
> 
> 1. Koresh and his people were basically minding their own business.
> 2. Some weapons violations may have been committed and I wouldn't have
>    disapproved of prosecuting him for those violations.  However, I think
>    the BATF was criminal for starting negotiations with a military style
>    assault and for firing into a house where there were children and other
>    noncombatants.
> 3. I don't see they couldn't just leave a token guard on the place and wait
>    the BDs out; I don't approve of the tear gas approach and,

In [12]:
target_names[newsgroups.target[0]]

'talk.politics.guns'

In [13]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(newsgroups.data,
                                                                    newsgroups.target,
                                                                    test_size=0.2,
                                                                    random_state=42)

In [14]:
newsgroups.target.shape

(2101,)

In [15]:
target_train.shape

(1680,)

In [16]:
target_test.shape

(421,)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(['I am teapot', 'Short and stout'])
vectorizer.get_feature_names()

['am', 'and', 'short', 'stout', 'teapot']

In [18]:
pd.DataFrame(data=vectorizer.transform(['I am a little teapot', 'Short and short']).toarray(),
            columns=vectorizer.get_feature_names())

,am,and,short,stout,teapot
0,1,0,0,0,1
1,0,1,2,0,0


In [43]:
tf_vectorizer = CountVectorizer()
tf_train = tf_vectorizer.fit_transform(data_train)
tf_train.shape

(1680, 29450)

In [44]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(tf_train, target_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [47]:
from sklearn import metrics

pred = clf.predict(tf_train)
metrics.accuracy_score(target_train, pred)

0.9964285714285714

In [48]:
pred = clf.predict(tf_vectorizer.transform(data_test))
metrics.accuracy_score(target_test, pred)

0.9738717339667459

In [23]:
def show_most_informative_features(classifier, vectorizer, categories):
    for i, c in enumerate(categories):
        print("%s: %s" % (c, ", ".join(np.asarray(vectorizer.get_feature_names())[np.argsort(classifier.coef_[i])[-10:]])))

In [49]:
show_most_informative_features(clf, tf_vectorizer, target_names)

comp.graphics: edu, from, it, for, is, in, of, and, to, the
rec.autos: for, you, that, is, it, in, and, of, to, the
talk.politics.guns: for, it, you, is, that, in, and, of, to, the
talk.religion.misc: not, it, you, is, in, that, and, to, of, the


In [62]:
from sklearn.pipeline import make_pipeline

model1 = make_pipeline(
    CountVectorizer(stop_words='english'),
    MultinomialNB())
model1.fit(data_train, target_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)), ('multinomialnb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [63]:
show_most_informative_features(
    model1.named_steps['multinomialnb'],
    model1.named_steps['countvectorizer'],
    target_names)

comp.graphics: host, posting, university, image, com, organization, subject, graphics, lines, edu
rec.autos: just, cars, writes, article, organization, lines, subject, com, car, edu
talk.politics.guns: guns, article, writes, organization, lines, subject, people, com, gun, edu
talk.religion.misc: article, writes, lines, organization, jesus, subject, people, god, com, edu


In [64]:
model1.predict(['foo'])

array([0])

In [25]:
def predict(classifier, vectorizer, categories, doc):
    return categories[classifier.predict(vectorizer.transform([doc]))[0]]

In [26]:
tf_vectorizer = CountVectorizer(
    stop_words='english'
)
tf_train = tf_vectorizer.fit_transform(data_train)
clf = MultinomialNB()
clf.fit(tf_train, target_train)
show_most_informative_features(clf, tf_vectorizer, target_names)

comp.graphics: host, posting, university, image, com, organization, subject, graphics, lines, edu
rec.autos: just, cars, writes, article, organization, lines, subject, com, car, edu
talk.politics.guns: guns, article, writes, organization, lines, subject, people, com, gun, edu
talk.religion.misc: article, writes, lines, organization, jesus, subject, people, god, com, edu


In [27]:
tf_test = tf_vectorizer.transform(data_test)
pred = clf.predict(tf_test)
metrics.accuracy_score(target_test, pred)

0.9762470308788599

In [28]:
newsgroups_clean = fetch_20newsgroups(categories=categories, remove=('headers', 'footers', 'quotes'))
print(newsgroups_clean.data[0])



  I think the legal term would be "negligent homicide"



  Well, it's nice to see someone with a brain, a general lack of paranoia, and
a willingness to put his thoughts in public.  I tend to agree with all you have
said.

  "Never assume foul motives when stupidity will do." -- Jim's Corrolary to 
						          Occam's Razor

semper fi,


In [29]:
data_train_clean, data_test_clean, target_train_clean, target_test_clean = train_test_split(
    newsgroups_clean.data,
    newsgroups_clean.target,
    test_size=0.2,
    random_state=42)

In [37]:
print(data_test_clean[0])
print(target_names[target_test_clean[0]])



There is a rite like this described in Joseph Campbell's
_Occidental_Mythology_.  He also described levels of initiation, I think
6?  I don't know where Campbell got his info, but I remember thinking he
was being a little eclectic.


Quite a bit.  If you haven't read Campbell, give him a try.  

talk.religion.misc


In [42]:
tf_test = tf_vectorizer.transform(data_test_clean)
pred = clf.predict(tf_test)
metrics.accuracy_score(target_test_clean, pred)

0.8741092636579573

In [40]:
tf_vectorizer = CountVectorizer(
    stop_words='english'
)
tf_train = tf_vectorizer.fit_transform(data_train_clean)
clf = MultinomialNB()
clf.fit(tf_train, target_train_clean)
show_most_informative_features(clf, tf_vectorizer, target_names)
tf_test = tf_vectorizer.transform(data_test_clean)
pred = clf.predict(tf_test)
metrics.accuracy_score(target_test_clean, pred)

comp.graphics: files, ftp, software, file, program, use, data, edu, image, graphics
rec.autos: time, think, engine, new, good, don, just, like, cars, car
talk.politics.guns: know, government, think, like, right, just, don, guns, people, gun
talk.religion.misc: know, bible, think, say, just, christian, don, jesus, people, god


0.8741092636579573

In [32]:
predict(clf, tf_vectorizer, target_names, 'a legal term for illegal car ftp software')

'rec.autos'

In [33]:
#import pyLDAvis
#import pyLDAvis.sklearn
#pyLDAvis.enable_notebook()